In [1]:
import pandas as pd
import datetime

In [25]:
class feature_engine:
    
    pd = __import__('pandas')
    datetime = __import__('datetime')
    
    def __init__(self):
        print('feature_engine V.0.1 \nImported pandas,datetime packages')
        
    def calc_price_change(self,tweet_time,financial_data,time_col_name,price_col_name,interval,amount,method='both'):
        #remove seconds to match financial data format
        tweet_time=tweet_time.replace(second=0)
        #default baseline is set for five minutes before tweet
        tweet_time_5min_before=tweet_time - self.datetime.timedelta(0,0,0,0,5)
        
        tweet_price=financial_data[financial_data[time_col_name]==tweet_time_5min_before][price_col_name]
        if len(tweet_price)==0:
            return float('NaN')
        tweet_price.reset_index(inplace=True,drop=True)

        def calculation(financial_data,time_col_name,new_time,price_col_name,tweet_price,direction):
            
            other_price=financial_data[financial_data[time_col_name]==new_time][price_col_name]
            other_price.reset_index(inplace=True,drop=True)
            if len(other_price)==0:
                return float('NaN')
            elif direction=='forward':
                return ((other_price.iloc[0]/tweet_price.iloc[0])-1)
            elif direction=='backward':
                return ((tweet_price.iloc[0]/other_price.iloc[0])-1)

        if (method=='both') or (method=='forward'):  
            if interval=='minutes':
                time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,amount)
            elif interval=='hours':
                time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,0,amount)
            elif interval=='days':
                time_forward=tweet_time + self.datetime.timedelta(amount,0,0,0,0)
            pct_change_forward=calculation(financial_data,time_col_name,time_forward,price_col_name,tweet_price,'forward')

        if (method=='both') or (method=='backward'): 
            if interval=='minutes':
                time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,amount)   
            elif interval=='hours':
                time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,0,amount) 
            elif interval=='days':
                time_backward=tweet_time - self.datetime.timedelta(amount,0,0,0,0)
            pct_change_backward=calculation(financial_data,time_col_name,time_backward,price_col_name,tweet_price,'backward')
        
        if method == 'both':
            return self.pd.DataFrame({f'{amount}_{interval}_forward_pct_change':pct_change_forward,
                          f'{amount}_{interval}_backward_pct_change':pct_change_backward},index=[0])
        elif method=='backward':
            return self.pd.DataFrame({f'{amount}_{interval}_backward_pct_change':pct_change_backward},index=[0])
        elif method=='forward':
            return self.pd.DataFrame({f'{amount}_{interval}_forward_pct_change':pct_change_forward},index=[0])
        
    def create_pricechg_columns(self,twitter_data,tweet_time_col_name,financial_data,fin_time_col_name,price_col_name,
                       interval_amount_dict,method='both'):
        from tqdm import tqdm
        new_columns=self.pd.DataFrame()
        for i, row in tqdm(twitter_data.iterrows()):
            temp_df=self.pd.DataFrame()
            for interval, amount in interval_amount_dict:
                new_df=self.calc_price_change(twitter_data[tweet_time_col_name].loc[i],financial_data,fin_time_col_name,
                                                   price_col_name,interval,amount,method)
                if not isinstance(new_df,float):
                    temp_df=self.pd.concat([temp_df,new_df],axis=1)
            for item in temp_df.columns:
                new_columns.at[i,item] = temp_df.loc[0,item]
        return new_columns
    
    def mean_encoding_tocolumn(self,features_df,categorical_column,target_column):
        features_df=features_df[[categorical_column,target_column]]
        grouped=features_df.groupby([categorical_column]).mean()
        features_df=features_df.merge(grouped,on=categorical_column)
        return features_df[f'{target_column}_y']
    
    def mean_encoding_todict(self,features_df,categorical_column,target_column):
        features_df=features_df[[categorical_column,target_column]]
        return features_df.groupby([categorical_column]).mean()
    
    def calculate_sum_volume(self,tweet_time,financial_data,time_col_name,volume_col_name,interval,amount,method='both'):
        #remove seconds to match financial data format
        tweet_time=tweet_time.replace(second=0)
        #default baseline is set for five minutes before tweet
        tweet_time_5min_before=tweet_time - self.datetime.timedelta(0,0,0,0,5)
        
#         tweet_volume=financial_data[financial_data[time_col_name]==tweet_time][volume_col_name]
#         if len(tweet_volume)==0:
#             return float('NaN')
#         tweet_volume.reset_index(inplace=True,drop=True)
        
        def calculation(financial_data,time_col_name,new_time,volume_col_name,tweet_time,direction):
            if direction=='forward':
                other_volume_df=financial_data.loc[(financial_data[time_col_name]<=new_time) & 
                                            (financial_data[time_col_name]>=tweet_time)][volume_col_name]
            elif direction=='backward':
                other_volume_df=financial_data.loc[(financial_data[time_col_name]>=new_time) & 
                                            (financial_data[time_col_name]<=tweet_time)][volume_col_name]
            other_volume=sum(other_volume_df[volume_col_name])
#             other_volume.reset_index(inplace=True,drop=True)
            if len(other_volume)==0:
                return float('NaN')
            else:
                return other_volume

        if (method=='both') or (method=='forward'):  
            if interval=='minutes':
                time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,amount)
            elif interval=='hours':
                time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,0,amount)
            elif interval=='days':
                time_forward=tweet_time + self.datetime.timedelta(amount,0,0,0,0)
            vol_sum_forward=calculation(financial_data,time_col_name,time_forward,volume_col_name,tweet_time_5min_before
                                        ,'forward')

        if (method=='both') or (method=='backward'): 
            if interval=='minutes':
                time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,amount)   
            elif interval=='hours':
                time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,0,amount) 
            elif interval=='days':
                time_backward=tweet_time - self.datetime.timedelta(amount,0,0,0,0)
            vol_sum_backward=calculation(financial_data,time_col_name,time_forward,volume_col_name,tweet_time,'backward')
        
        if method == 'both':
            return self.pd.DataFrame({f'{amount}_{interval}_forward_vol_sum':vol_sum_forward,
                          f'{amount}_{interval}_backward_vol_sum':vol_sum_backward},index=[0])
        elif method=='backward':
            return self.pd.DataFrame({f'{amount}_{interval}_backward_vol_sum':vol_sum_backward},index=[0])
        elif method=='forward':
            return self.pd.DataFrame({f'{amount}_{interval}_forward_vol_sum':vol_sum_forward},index=[0])
        
    def create_volumesum_columns(self,twitter_data,tweet_time_col_name,financial_data,fin_time_col_name,volume_col_name,
                       interval_amount_dict,method='both'):
        from tqdm import tqdm
        new_columns=self.pd.DataFrame()
        for i, row in tqdm(twitter_data.iterrows()):
            temp_df=self.pd.DataFrame()
            for interval, amount in interval_amount_dict:
                new_df=self.calculate_sum_volume(twitter_data[tweet_time_col_name].iloc[i],financial_data,fin_time_col_name,
                                                   volume_col_name,interval,amount)
                if not isinstance(new_df,float):
                    temp_df=self.pd.concat([temp_df,new_df],axis=1)
            for item in temp_df.columns:
                new_columns.at[i,item] = temp_df.loc[0,item]
        return new_columns

In [10]:
financial_data=pd.read_csv('04052009-18122019-SPYDIA-minutedata.csv',parse_dates=['DATETIME'])
twitter_data=pd.read_csv('trump_tweets_reducted_after_nlp.csv',index_col=0,parse_dates=['created_at_utc'])

In [11]:
sp500_data=financial_data[financial_data['SYM_ROOT']=='SPY']
dow_data=financial_data[financial_data['SYM_ROOT']=='DIA']

In [26]:
feat_eng = feature_engine()

feature_engine V.0.1 
Imported pandas,datetime packages


In [ ]:
tweets_finance=feat_eng.create_pricechg_columns(twitter_data,'created_at_utc',sp500_data,'DATETIME','PRICE',
                       [('minutes',1),('minutes',5),('minutes',10),('minutes',15),('minutes',30),('hours',1),
                        ('hours',3),('hours',6),('days',1),('days',5),('days',10),('days',30)],method='forward')




0it [00:00, ?it/s]


1it [00:00,  2.72it/s]


2it [00:00,  3.39it/s]


3it [00:00,  3.59it/s]


4it [00:00,  4.34it/s]


5it [00:01,  4.23it/s]


6it [00:01,  4.92it/s]


7it [00:01,  5.67it/s]


8it [00:01,  4.81it/s]


9it [00:01,  4.40it/s]


10it [00:02,  5.08it/s]


11it [00:02,  5.56it/s]


12it [00:02,  5.05it/s]


13it [00:02,  4.75it/s]


14it [00:02,  4.46it/s]


15it [00:03,  4.98it/s]


16it [00:03,  5.40it/s]


17it [00:03,  4.93it/s]


18it [00:03,  5.64it/s]


19it [00:03,  5.07it/s]


20it [00:04,  4.36it/s]


21it [00:04,  4.89it/s]


22it [00:04,  5.46it/s]


23it [00:04,  4.94it/s]


24it [00:04,  5.66it/s]


25it [00:04,  6.15it/s]


26it [00:05,  6.57it/s]


27it [00:05,  5.20it/s]


28it [00:05,  5.72it/s]


29it [00:05,  5.04it/s]


30it [00:05,  5.66it/s]


31it [00:06,  4.64it/s]


32it [00:06,  4.31it/s]


33it [00:06,  5.06it/s]


34it [00:06,  4.45it/s]


35it [00:07,  4.25it/s]


36it [00:07,  4.20it/s]


37it [00:07,  4.06it/s]


38it [00:07,  3.75it/s]

307it [01:11,  4.24it/s]


308it [01:11,  4.21it/s]


309it [01:12,  4.96it/s]


310it [01:12,  4.66it/s]


311it [01:12,  4.49it/s]


312it [01:12,  4.42it/s]


313it [01:12,  5.21it/s]


314it [01:13,  4.89it/s]


315it [01:13,  4.67it/s]


316it [01:13,  4.53it/s]


317it [01:13,  5.32it/s]


318it [01:13,  6.04it/s]


319it [01:13,  6.68it/s]


320it [01:14,  5.41it/s]


321it [01:14,  5.85it/s]


322it [01:14,  4.90it/s]


323it [01:14,  4.36it/s]


324it [01:15,  4.28it/s]


325it [01:15,  5.09it/s]


326it [01:15,  4.80it/s]


327it [01:15,  4.63it/s]


328it [01:15,  5.38it/s]


329it [01:16,  4.84it/s]


330it [01:16,  5.60it/s]


331it [01:16,  5.10it/s]


332it [01:16,  4.84it/s]


333it [01:16,  4.61it/s]


334it [01:17,  4.37it/s]


335it [01:17,  4.33it/s]


336it [01:17,  3.95it/s]


337it [01:17,  3.86it/s]


338it [01:18,  3.93it/s]


339it [01:18,  3.98it/s]


340it [01:18,  4.06it/s]


341it [01:18,  4.86it/s]


342it [01:19,  4.65it/s]


343it [01:19,  4.51it/s]


3

610it [02:13,  5.32it/s]


611it [02:14,  4.72it/s]


612it [02:14,  4.40it/s]


613it [02:14,  5.19it/s]


614it [02:14,  5.96it/s]


615it [02:14,  5.31it/s]


616it [02:15,  4.88it/s]


617it [02:15,  4.65it/s]


618it [02:15,  4.52it/s]


619it [02:15,  4.45it/s]


620it [02:16,  4.35it/s]


621it [02:16,  4.33it/s]


622it [02:16,  4.31it/s]


623it [02:16,  4.29it/s]


624it [02:16,  5.06it/s]


625it [02:17,  4.79it/s]


626it [02:17,  4.64it/s]


627it [02:17,  4.50it/s]


628it [02:17,  4.43it/s]


629it [02:18,  4.36it/s]


630it [02:18,  4.29it/s]


631it [02:18,  4.27it/s]


632it [02:18,  4.21it/s]


633it [02:19,  4.17it/s]


634it [02:19,  4.16it/s]


635it [02:19,  4.12it/s]


636it [02:19,  4.08it/s]


637it [02:20,  3.97it/s]


638it [02:20,  3.95it/s]


639it [02:20,  4.66it/s]


640it [02:20,  5.41it/s]


641it [02:20,  4.89it/s]


642it [02:20,  5.60it/s]


643it [02:21,  5.00it/s]


644it [02:21,  4.70it/s]


645it [02:21,  5.43it/s]


646it [02:21,  6.13it/s]


6

913it [03:09,  4.69it/s]


914it [03:10,  4.52it/s]


915it [03:10,  4.38it/s]


916it [03:10,  4.35it/s]


917it [03:10,  5.13it/s]


918it [03:10,  4.84it/s]


919it [03:11,  5.62it/s]


920it [03:11,  5.12it/s]


921it [03:11,  4.82it/s]


922it [03:11,  5.57it/s]


923it [03:11,  6.31it/s]


924it [03:11,  5.48it/s]


925it [03:12,  5.01it/s]


926it [03:12,  4.75it/s]


927it [03:12,  5.19it/s]


928it [03:12,  5.49it/s]


929it [03:13,  4.68it/s]


930it [03:13,  4.55it/s]


931it [03:13,  5.36it/s]


932it [03:13,  6.09it/s]


933it [03:13,  6.78it/s]


934it [03:13,  7.34it/s]


935it [03:13,  6.04it/s]


936it [03:14,  5.07it/s]


937it [03:14,  5.57it/s]


938it [03:14,  6.01it/s]


939it [03:14,  6.59it/s]


940it [03:14,  7.18it/s]


941it [03:14,  5.95it/s]


942it [03:15,  6.63it/s]


943it [03:15,  5.66it/s]


944it [03:15,  5.16it/s]


945it [03:15,  5.91it/s]


946it [03:15,  5.29it/s]


947it [03:16,  4.83it/s]


948it [03:16,  5.59it/s]


949it [03:16,  6.32it/s]


9

1208it [04:12,  5.92it/s]


1209it [04:12,  4.98it/s]


1210it [04:13,  4.71it/s]


1211it [04:13,  5.49it/s]


1212it [04:13,  6.24it/s]


1213it [04:13,  6.86it/s]


1214it [04:13,  5.72it/s]


1215it [04:14,  4.95it/s]


1216it [04:14,  5.52it/s]


1217it [04:14,  4.87it/s]


1218it [04:14,  4.64it/s]


1219it [04:14,  4.43it/s]


1220it [04:15,  4.30it/s]


1221it [04:15,  4.28it/s]


1222it [04:15,  4.29it/s]


1223it [04:15,  4.28it/s]


1224it [04:16,  4.27it/s]


1225it [04:16,  5.04it/s]


1226it [04:16,  4.80it/s]


1227it [04:16,  5.59it/s]


1228it [04:16,  6.31it/s]


1229it [04:16,  6.90it/s]


1230it [04:16,  7.45it/s]


1231it [04:17,  7.82it/s]


1232it [04:17,  8.11it/s]


1233it [04:17,  8.37it/s]


1234it [04:17,  6.46it/s]


1235it [04:17,  5.65it/s]


1236it [04:17,  5.15it/s]


1237it [04:18,  5.88it/s]


1238it [04:18,  5.24it/s]


1239it [04:18,  4.93it/s]


1240it [04:18,  4.62it/s]


1241it [04:19,  4.47it/s]


1242it [04:19,  4.38it/s]


1243it [04:19,  4.29

...

7048it [22:58,  5.78it/s]


7049it [22:58,  6.10it/s]


7050it [22:58,  4.82it/s]


7051it [22:59,  4.91it/s]


7052it [22:59,  4.20it/s]


7053it [22:59,  4.76it/s]


7054it [22:59,  5.23it/s]


7055it [22:59,  5.60it/s]


7056it [23:00,  5.83it/s]


7057it [23:00,  4.75it/s]


7058it [23:00,  3.75it/s]


7059it [23:01,  3.42it/s]


7060it [23:01,  3.39it/s]


7061it [23:01,  4.02it/s]


7062it [23:01,  4.23it/s]


7063it [23:02,  3.59it/s]


7064it [23:02,  3.45it/s]


7065it [23:02,  3.30it/s]


7066it [23:03,  3.33it/s]


7067it [23:03,  3.34it/s]


7068it [23:03,  3.33it/s]


7069it [23:04,  3.27it/s]


7070it [23:04,  3.33it/s]


7071it [23:04,  3.33it/s]


7072it [23:04,  3.98it/s]


7073it [23:05,  3.76it/s]


7074it [23:05,  4.40it/s]


7075it [23:05,  4.97it/s]


7076it [23:05,  5.45it/s]


7077it [23:05,  4.57it/s]


7078it [23:05,  5.07it/s]


7079it [23:06,  5.59it/s]


7080it [23:06,  5.95it/s]


7081it [23:06,  6.21it/s]


7082it [23:06,  6.47it/s]


7083it [23:06,  6.66

7340it [23:59,  4.37it/s]


7341it [23:59,  3.46it/s]


7342it [24:00,  3.40it/s]


7343it [24:00,  3.42it/s]


7344it [24:00,  4.09it/s]


7345it [24:00,  4.56it/s]


7346it [24:00,  4.98it/s]


7347it [24:01,  4.06it/s]


7348it [24:01,  4.62it/s]


7349it [24:01,  4.12it/s]


7350it [24:02,  3.45it/s]


7351it [24:02,  2.90it/s]


7352it [24:02,  2.90it/s]


7353it [24:03,  3.00it/s]


7354it [24:03,  3.07it/s]


7355it [24:03,  2.95it/s]


7356it [24:04,  3.55it/s]


7357it [24:04,  3.53it/s]


7358it [24:04,  3.82it/s]


7359it [24:04,  3.63it/s]


7360it [24:05,  3.53it/s]


7361it [24:05,  3.42it/s]


7362it [24:05,  3.19it/s]


7363it [24:06,  3.21it/s]


7364it [24:06,  3.22it/s]


7365it [24:06,  3.13it/s]


7366it [24:06,  3.48it/s]


7367it [24:07,  3.34it/s]


7368it [24:07,  3.27it/s]


7369it [24:07,  3.27it/s]


7370it [24:08,  3.24it/s]


7371it [24:08,  2.98it/s]


7372it [24:08,  3.02it/s]


7373it [24:09,  3.07it/s]


7374it [24:09,  3.06it/s]


7375it [24:09,  2.90

7632it [25:09,  5.71it/s]


7633it [25:09,  6.08it/s]


7634it [25:09,  4.45it/s]


7635it [25:09,  3.84it/s]


7636it [25:10,  3.60it/s]


7637it [25:10,  3.48it/s]


7638it [25:10,  3.40it/s]


7639it [25:11,  4.01it/s]


7640it [25:11,  4.58it/s]


7641it [25:11,  5.11it/s]


7642it [25:11,  3.52it/s]


7643it [25:12,  3.30it/s]


7644it [25:12,  3.04it/s]


7645it [25:12,  3.01it/s]


7646it [25:13,  3.63it/s]


7647it [25:13,  4.26it/s]


7648it [25:13,  4.82it/s]


7649it [25:13,  5.29it/s]


7650it [25:13,  5.79it/s]


7651it [25:13,  6.19it/s]


7652it [25:13,  6.39it/s]


7653it [25:14,  6.43it/s]


7654it [25:14,  6.35it/s]


7655it [25:14,  4.73it/s]


7656it [25:14,  4.16it/s]


7657it [25:15,  3.87it/s]


7658it [25:15,  3.72it/s]


7659it [25:15,  3.59it/s]


7660it [25:15,  4.21it/s]


7661it [25:16,  3.58it/s]


7662it [25:16,  3.36it/s]


7663it [25:16,  3.33it/s]


7664it [25:17,  3.30it/s]


7665it [25:17,  3.22it/s]


7666it [25:17,  2.93it/s]


7667it [25:18,  3.53

7924it [26:13,  6.67it/s]


7925it [26:14,  6.49it/s]


7926it [26:14,  6.37it/s]


7927it [26:14,  6.40it/s]


7928it [26:14,  6.50it/s]


7929it [26:14,  6.60it/s]


7930it [26:14,  5.05it/s]


7931it [26:15,  4.18it/s]


7932it [26:15,  3.86it/s]


7933it [26:15,  3.68it/s]


7934it [26:16,  3.52it/s]


7935it [26:16,  3.48it/s]


7936it [26:16,  3.41it/s]


7937it [26:17,  3.35it/s]


7938it [26:17,  3.15it/s]


7939it [26:17,  2.97it/s]


7940it [26:18,  2.82it/s]


7941it [26:18,  2.71it/s]


7942it [26:19,  2.85it/s]


7943it [26:19,  3.48it/s]


7944it [26:19,  3.40it/s]


7945it [26:19,  4.03it/s]


7946it [26:19,  3.75it/s]


7947it [26:20,  4.36it/s]


7948it [26:20,  4.90it/s]


7949it [26:20,  5.40it/s]


7950it [26:20,  5.80it/s]


7951it [26:20,  6.11it/s]


7952it [26:20,  6.46it/s]


7953it [26:20,  6.65it/s]


7954it [26:21,  6.72it/s]


7955it [26:21,  6.74it/s]


7956it [26:21,  6.80it/s]


7957it [26:21,  5.23it/s]


7958it [26:21,  5.67it/s]


7959it [26:21,  6.05

8216it [27:25,  6.42it/s]


8217it [27:25,  6.65it/s]


8218it [27:25,  6.53it/s]

In [ ]:
tweets_finance.to_csv('tweets_finance.csv',index=True)